# Fourth trial at displaying a coal seam using a PolyData

We want to work on the first polygon for now...

In [1]:
import geopandas as gpd
import pyvista as pv
import pyacvd
from shapely.geometry import Polygon
import numpy as np

In [2]:
gdf = gpd.read_file('../../../../data/example.gpkg', layer = 'Parties exploitées - Caillette')

In [3]:
gdf.head()

,Année,geometry
0,1941,"POLYGON Z ((154613.672 123991.170 -663.000, 15..."
1,1942,"POLYGON Z ((154816.546 123955.316 -671.100, 15..."
2,1939,"POLYGON Z ((154912.246 123824.192 -686.800, 15..."
3,1942,"POLYGON Z ((154868.618 123744.141 -698.900, 15..."
4,1938,"POLYGON Z ((154949.664 123925.004 -679.300, 15..."


In [20]:
remesh_blocks = pv.MultiBlock()
polygon_blocks = pv.MultiBlock()
for idx, row in gdf.iterrows():
    if isinstance(row.geometry, Polygon):
                    number_of_vertices = len(list(row.geometry.exterior.coords))
                    if row.geometry.has_z:  # TODO: Can we handle Polygons with holes?
                        vertices = list(row.geometry.exterior.coords)
                    else:
                        vertices = zip(row.geometry.exterior.xy[0], row.geometry.exterior.xy[1],
                                       itertools.repeat(row[elevation]))
    vertices = np.array(vertices)
    face = [number_of_vertices-1]
    face.extend([i for i in range(number_of_vertices-1)])
    face = np.array(face)
    polygon = pv.PolyData(vertices, face)
    polygon.point_data['Elevation'] = vertices.T[2]
    polygon_d = polygon.delaunay_2d()
    polygon_d = polygon_d.subdivide(3, subfilter='linear')
    clipped = polygon_d.clip_surface(polygon.extrude([0.,0.,500.]).transform(np.array([[1.,0.,0.,0.],[0.,1.,0.,0.],[0.,0.,1.,-250.],[0.,0.,0.,1.]])), invert=False).decimate_pro(0.75)
    cluster = pyacvd.Clustering(polygon_d)
    # mesh is not dense enough for uniform remeshing
    cluster.cluster(20000, maxiter=2000)
    remesh = cluster.create_mesh()
    remesh = remesh.clip_surface(polygon.extrude([0.,0.,500.]).transform(np.array([[1.,0.,0.,0.],[0.,1.,0.,0.],[0.,0.,1.,-250.],[0.,0.,0.,1.]])), invert=False).clean().decimate_pro(0.75).smooth(n_iter=100000, boundary_smoothing=False)
    remesh.point_data['Elevation'] = remesh.points[:,2]
    remesh_blocks.append(remesh)
    polygon_blocks.append(polygon_d.smooth(n_iter=20000))

/home/su530201/.local/share/virtualenvs/gsdma_2021-CLBGPK-k/lib/python3.8/site-packages/pyvista/core/dataset.py:1192: PyvistaDeprecationWarning: Use of `point_arrays` is deprecated. Use `point_data` instead.
  warnings.warn(
/home/su530201/.local/share/virtualenvs/gsdma_2021-CLBGPK-k/lib/python3.8/site-packages/pyvista/core/dataset.py:1332: PyvistaDeprecationWarning: Use of `cell_arrays` is deprecated. Use `cell_data` instead.
  warnings.warn(


## Interactive representation

In [16]:
dargs = dict(scalars='Elevation', cmap='rainbow', show_edges=True, window_size=(1024,480))

remesh_blocks.plot(jupyter_backend='panel', color='w', style='surface', interpolate_before_map=True,
        scalar_bar_args={'title': 'Elevation'}, **dargs)

In [19]:
dargs = dict(scalars='Elevation', cmap='rainbow', show_edges=True, window_size=(1024,480))

polygon_blocks.plot(jupyter_backend='panel', color='w', style='surface', interpolate_before_map=True,
        scalar_bar_args={'title': 'Elevation'}, **dargs)